In [1]:
import pandas
import geopandas 
import matplotlib.pyplot as plt
from   matplotlib.pyplot import imread
import matplotlib.patches as mpatches
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeat
#import cartopy.io.shapereader as shpreader
from   cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import shapely.geometry as sgeom
from   shapely.geometry import LineString
from   shapely.geometry import MultiPoint
from   shapely.geometry import Point, Polygon

%matplotlib inline

In [8]:
import urllib
import requests
from zipfile import ZipFile
import datetime 



In [41]:
file_name = 'gtwo_shapefiles.zip'
url = f'https://www.nhc.noaa.gov/xgtwo/{file_name}'
 
# download the file contents in binary format
r = requests.get(url)
 
# open method to open a file on your system and write the contents
with open(f'nhc_latest/{file_name}', 'wb') as code:
    code.write(r.content)

In [42]:
with ZipFile(f'nhc_latest/{file_name}', 'r') as gtwo_zip:
    #print(gtwo_zip.namelist())
    gtwo_areas = gtwo_zip.namelist()[2]
    gtwo_lines = gtwo_zip.namelist()[7]
    gtwo_points = gtwo_zip.namelist()[12]

In [45]:
gtwo_areas_gdf  = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{gtwo_areas}')
gtwo_lines_gdf  = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{gtwo_lines}')
gtwo_points_gdf = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{gtwo_points}')

In [48]:
gtwo_points_gdf

,BASIN,AREA,PROB2DAY,RISK2DAY,PROB5DAY,RISK5DAY,geometry
0,Atlantic,1,20%,Low,20%,Low,POINT (-77.64000 12.09000)


In [49]:
current_year = datetime.date.today().year
storm_number = '12'
file_name    = f'al{storm_number}{current_year}_5day_latest.zip'
#ex.: https://www.nhc.noaa.gov/gis/forecast/archive/al122021_5day_latest.zip
url       = f'https://www.nhc.noaa.gov/gis/forecast/archive/{file_name}'
 
# download the file contents in binary format
r = requests.get(url)
 
# open method to open a file on your system and write the contents
with open(f'nhc_latest/{file_name}', 'wb') as code:
    code.write(r.content)

In [60]:
with ZipFile(f'nhc_latest/{file_name}', 'r') as aft_cu_zip:
    #print(aft_cu_zip.namelist())
    track_line  = aft_cu_zip.namelist()[2]
    cone        = aft_cu_zip.namelist()[7]
    points      = aft_cu_zip.namelist()[12]
    
track_line_gdf  = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{track_line}')
cone_gdf        = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{cone}')
points_gdf      = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{points}')

['al122021-002_5day_lin.dbf', 'al122021-002_5day_lin.prj', 'al122021-002_5day_lin.shp', 'al122021-002_5day_lin.shp.xml', 'al122021-002_5day_lin.shx', 'al122021-002_5day_pgn.dbf', 'al122021-002_5day_pgn.prj', 'al122021-002_5day_pgn.shp', 'al122021-002_5day_pgn.shp.xml', 'al122021-002_5day_pgn.shx', 'al122021-002_5day_pts.dbf', 'al122021-002_5day_pts.prj', 'al122021-002_5day_pts.shp', 'al122021-002_5day_pts.shp.xml', 'al122021-002_5day_pts.shx']


In [61]:
cone_gdf

,STORMNAME,STORMTYPE,ADVDATE,ADVISNUM,STORMNUM,FCSTPRD,BASIN,geometry
0,Twelve,TD,200 AM CVT Wed Sep 01 2021,2,12.0,120.0,AL,"POLYGON ((-23.42385 11.85572, -23.39655 11.852..."


In [62]:
Advisory_Forecast_Track_Cone_of_Uncertainty_and_Watches_Warnings = 'https://www.nhc.noaa.gov/gis/forecast/archive/al122021_5day_latest.zip'
Advisory_Wind_Field_and_Forecast_Wind_Radii                      = 'https://www.nhc.noaa.gov/gis/forecast/archive/al122021_fcst_latest.zip'
Preliminary_Best_Track                                           = 'https://www.nhc.noaa.gov/gis/best_track/al122021_best_track.zip'
Graphical_Tropical_Weather_Outlook                               = 'https://www.nhc.noaa.gov/xgtwo/gtwo_shapefiles.zip'
Wind_Speed_Probabilities_Points                                  = 'https://www.nhc.noaa.gov/gis/forecast/archive/wsp_120hrhalfDeg_latest.zip'
Wind_Speed_Probabilities_Polygons                                = 'https://www.nhc.noaa.gov/gis/forecast/archive/wsp_120hr5km_latest.zip'


In [15]:
storm_number = '12'
current_year = datetime.date.today().year
file_names   = [f'al{storm_number}{current_year}_5day_latest.zip',
                f'al{storm_number}{current_year}_fcst_latest.zip',
                f'al{storm_number}{current_year}_best_track.zip',
                f'gtwo_shapefiles.zip',
                f'wsp_120hrhalfDeg_latest.zip',
                f'wsp_120hr5km_latest.zip'
               ]

urls        = [f'https://www.nhc.noaa.gov/gis/forecast/archive',
               f'https://www.nhc.noaa.gov/gis/forecast/archive',
               f'https://www.nhc.noaa.gov/gis/best_track',
               f'https://www.nhc.noaa.gov/xgtwo',
               f'https://www.nhc.noaa.gov/gis/forecast/archive',
               f'https://www.nhc.noaa.gov/gis/forecast/archive'
              ]

In [24]:
for idx, file_name in enumerate(file_names):
    url = f'{urls[idx]}/{file_name}'
    
    # download the file contents in binary format
    r = requests.get(url)
    # open method to open a file on your system and write the contents
    with open(f'nhc_latest/{file_name}', 'wb') as code:
        code.write(r.content)
        
    with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
        print(zip_file_name.namelist())
        

['al122021-005_5day_lin.dbf', 'al122021-005_5day_lin.prj', 'al122021-005_5day_lin.shp', 'al122021-005_5day_lin.shp.xml', 'al122021-005_5day_lin.shx', 'al122021-005_5day_pgn.dbf', 'al122021-005_5day_pgn.prj', 'al122021-005_5day_pgn.shp', 'al122021-005_5day_pgn.shp.xml', 'al122021-005_5day_pgn.shx', 'al122021-005_5day_pts.dbf', 'al122021-005_5day_pts.prj', 'al122021-005_5day_pts.shp', 'al122021-005_5day_pts.shp.xml', 'al122021-005_5day_pts.shx']
['al122021_2021090121_forecastradii.shp', 'al122021_2021090121_forecastradii.shx', 'al122021_2021090121_forecastradii.dbf', 'al122021_2021090121_forecastradii.prj', 'al122021_2021090121_forecastradii.shp.xml', 'al122021_2021090121_initialradii.shp', 'al122021_2021090121_initialradii.shx', 'al122021_2021090121_initialradii.dbf', 'al122021_2021090121_initialradii.prj', 'al122021_2021090121_initialradii.shp.xml']
['AL122021_pts.dbf', 'AL122021_pts.prj', 'AL122021_pts.shp', 'AL122021_pts.shp.xml', 'AL122021_pts.shx', 'AL122021_lin.dbf', 'AL122021_lin

In [26]:
with ZipFile(f'nhc_latest/al122021_fcst_latest.zip', 'r') as zip_file:
    print(zip_file.namelist())

['al122021_2021090121_forecastradii.shp', 'al122021_2021090121_forecastradii.shx', 'al122021_2021090121_forecastradii.dbf', 'al122021_2021090121_forecastradii.prj', 'al122021_2021090121_forecastradii.shp.xml', 'al122021_2021090121_initialradii.shp', 'al122021_2021090121_initialradii.shx', 'al122021_2021090121_initialradii.dbf', 'al122021_2021090121_initialradii.prj', 'al122021_2021090121_initialradii.shp.xml']


In [28]:
'fix' in 'hola.fix'

True